In [ ]:
!pip install torch torchvision torchaudio tqdm numpy opencv-python scikit-learn albumentations torchmetrics



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 54.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
data_path = "/content/drive/MyDrive/Shop DataSet"


In [ ]:
import zipfile
zip_path = "/content/drive/MyDrive/Shop DataSet.zip"
extract_path = "/content/shop_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
import os
from collections import Counter


data_root = "/content/shop_dataset/Shop DataSet"


classes = sorted([d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))])
print("Classes:", classes)


def count_videos_per_class(root):
    counter = {}
    for cls in classes:
        cls_path = os.path.join(root, cls)
        vids = [f for f in os.listdir(cls_path) if f.lower().endswith(('.mp4', '.avi'))]
        counter[cls] = len(vids)
    return counter

print(count_videos_per_class(data_root))



Classes: ['non shop lifters', 'shop lifters']
{'non shop lifters': 531, 'shop lifters': 324}


In [ ]:

import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import VideoMAEFeatureExtractor, VideoMAEForVideoClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from collections import Counter


In [ ]:
from transformers import VideoMAEForVideoClassification
feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base-finetuned-kinetics",
    num_labels=2,
    ignore_mismatched_sizes=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:30: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base-finetuned-kinetics and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class HuggingFaceVideoDataset(Dataset):
    def __init__(self, video_paths, labels, num_frames=16, feature_extractor=None):
        self.video_paths = video_paths
        self.labels = labels
        self.num_frames = num_frames
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        path = self.video_paths[idx]
        label = self.labels[idx]

        cap = cv2.VideoCapture(path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
        cap.release()

        indices = np.linspace(0, len(frames)-1, self.num_frames, dtype=int)
        sampled_frames = [frames[i] for i in indices]

        return {"pixel_values": sampled_frames, "labels": label}


In [ ]:
def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    labels = torch.tensor([item["labels"] for item in batch])
    inputs = feature_extractor(pixel_values, return_tensors="pt")
    inputs["labels"] = labels
    return inputs

In [ ]:
import os
import glob


class_names = sorted([d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))])
print("Classes found:", class_names)

video_paths = []
labels = []

label_map = {name: idx for idx, name in enumerate(class_names)}

for cls in class_names:
    cls_folder = os.path.join(data_root, cls)
    videos = glob.glob(os.path.join(cls_folder, "*"))
    for v in videos:
        if v.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
            video_paths.append(v)
            labels.append(label_map[cls])

print(f"Total videos: {len(video_paths)}")
print(f"Labels count: {len(labels)}")


Classes found: ['non shop lifters', 'shop lifters']
Total videos: 855
Labels count: 855


In [ ]:
from sklearn.model_selection import train_test_split


train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    video_paths, labels, test_size=0.3, stratify=labels, random_state=42
)

val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)


In [ ]:
train_dataset = HuggingFaceVideoDataset(train_paths, train_labels, feature_extractor=feature_extractor)
val_dataset = HuggingFaceVideoDataset(val_paths, val_labels, feature_extractor=feature_extractor)
test_dataset = HuggingFaceVideoDataset(test_paths, test_labels, feature_extractor=feature_extractor)


In [ ]:

class_names = ["non shop lifters", "shop lifters"]
label_map = {name: idx for idx, name in enumerate(class_names)}

video_paths = []
labels = []
for cls in class_names:
    cls_dir = os.path.join(data_root, cls)
    for video_file in os.listdir(cls_dir):
        if video_file.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
            video_paths.append(os.path.join(cls_dir, video_file))
            labels.append(label_map[cls])

print(f"Total videos found: {len(video_paths)}")
print(f"Class distribution: {Counter(labels)}")


Total videos found: 855
Class distribution: Counter({0: 531, 1: 324})


In [ ]:
!pip install --upgrade transformers

In [ ]:
# Data partitioning
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    video_paths, labels, test_size=0.3, stratify=labels, random_state=42
)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)
# Preparing datasets
train_dataset = HuggingFaceVideoDataset(train_paths, train_labels, feature_extractor=feature_extractor)
val_dataset = HuggingFaceVideoDataset(val_paths, val_labels, feature_extractor=feature_extractor)
test_dataset = HuggingFaceVideoDataset(test_paths, test_labels, feature_extractor=feature_extractor)
# TrainingArguments setup
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
)

# Start training

trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: duaaswalmeh5 (duaaswalmeh5-yarmouk-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.755900
20,0.731100
30,0.238800
40,0.399300
50,0.551700
60,0.372000
70,0.189200
80,0.776400
90,0.307100
100,0.167200


TrainOutput(global_step=1495, training_loss=0.06418247536852364, metrics={'train_runtime': 1180.0261, 'train_samples_per_second': 2.534, 'train_steps_per_second': 1.267, 'total_flos': 3.7257331225205146e+18, 'train_loss': 0.06418247536852364, 'epoch': 5.0})

In [ ]:

# After training you can evaluate the model on test_dataset
results = trainer.evaluate(test_dataset)
print(results)
# Start training
trainer.train()

# After training you can evaluate the model on test_dataset
results = trainer.evaluate(test_dataset)
print(results)

{'eval_loss': 2.9623741284012794e-05, 'eval_runtime': 44.2674, 'eval_samples_per_second': 2.914, 'eval_steps_per_second': 1.468, 'epoch': 5.0}


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


{'eval_loss': 5.936355591984466e-06, 'eval_runtime': 42.0563, 'eval_samples_per_second': 3.067, 'eval_steps_per_second': 1.546, 'epoch': 5.0}
